# Lab 3B: きめ細かいアクセス制御を持つ修復エージェント

## 概要
この Lab は、JWT 認証と Lambda インターセプターを使用したロールベースのアクセス制御を導入することで Lab 3A を拡張します。SRE ユーザーが修復を計画でき、承認者のみが実行できる職務分離パターンを実装します。

## 学習目標
- Cognito による JWT ベース認証の理解
- Lambda インターセプターによるロールベースアクセス制御（RBAC）の実装
- カスタム認可を持つ AgentCore Gateway のデプロイと設定
- 自動修復ワークフローでの職務分離の強制
- アクセス決定の監視と監査

## ペルソナとアクセスモデル

### SRE ロール
- **権限:** 修復プランの作成と検証
- **制限:** 修復の実行不可
- **ユースケース:** 問題の診断と修正の提案

### Approver ロール
- **権限:** 修復の実行と検証
- **制限:** なし（フルアクセス）
- **ユースケース:** SRE プランのレビューと承認された修正の実行

---

# ステップ 1: ユースケースの説明

## シナリオ

組織でインフラストラクチャ変更に **職務分離** が必要です:

- **SRE チーム:** 問題を診断し、修復プランを作成
- **Approver チーム:** 承認された変更をレビューして実行

**問題:** アクセス制御がないと、誰でもインフラストラクチャ変更を実行でき、コンプライアンスとセキュリティリスクが発生します。

**解決策:** JWT 認証 + Lambda インターセプターを使用して、Gateway レベルでロールベースの権限を強制します。

## なぜ JWT + Lambda インターセプター？

**JWT 認証:**
- 業界標準のトークン形式
- ユーザー ID とグループメンバーシップを含む
- Cognito によって暗号署名
- データベース検索不要

**Lambda インターセプター:**
- Gateway の REQUEST フェーズで実行
- Runtime にルーティングする前に JWT クレームを検査
- きめ細かいアクセス制御を強制
- CloudWatch ログによる監査証跡を提供

**メリット:**
- 一元化された認可ロジック
- エージェントコードの変更不要
- スケーラブルでサーバーレス
- 監査と変更が容易

## ワークフロー図

```
┌─────────────────────────────────────────────────────────────────┐
│                    SRE ワークフロー（計画）                       │
└─────────────────────────────────────────────────────────────────┘

SRE ユーザー → Cognito 認証 → JWT（groups: ["sre"]）
                                ↓
                          Gateway がトークンを受信
                                ↓
                    Lambda インターセプターがチェック:
                    - cognito:groups = ["sre"] を抽出
                    - ツール: generate_remediation_plan
                    - 権限: ✅ 許可
                                ↓
                          Runtime が実行
                                ↓
                    修復プランが作成 ✅

┌─────────────────────────────────────────────────────────────────┐
│              SRE ワークフロー（実行の試行）                       │
└─────────────────────────────────────────────────────────────────┘

SRE ユーザー → Cognito 認証 → JWT（groups: ["sre"]）
                                ↓
                          Gateway がトークンを受信
                                ↓
                    Lambda インターセプターがチェック:
                    - cognito:groups = ["sre"] を抽出
                    - ツール: execute_remediation_step
                    - 権限: ❌ 拒否
                                ↓
                    Gateway でリクエストがブロック ❌

┌─────────────────────────────────────────────────────────────────┐
│                 Approver ワークフロー（実行）                     │
└─────────────────────────────────────────────────────────────────┘

Approver → Cognito 認証 → JWT（groups: ["approvers"]）
                                ↓
                          Gateway がトークンを受信
                                ↓
                    Lambda インターセプターがチェック:
                    - cognito:groups = ["approvers"] を抽出
                    - ツール: execute_remediation_step
                    - 権限: ✅ 許可
                                ↓
                          Runtime が実行
                                ↓
                    修復が実行 ✅
```

In [ ]:
%pip install -q -r requirements.txt
print("✅ ワークショップの依存関係がインストールされました")

In [ ]:
# Lab 3A の完了を確認
from lab_helpers.parameter_store import get_parameter
from lab_helpers.constants import PARAMETER_PATHS

try:
    user_pool_id = get_parameter(PARAMETER_PATHS['cognito']['user_pool_id'])
    print("✅ Lab 3A の前提条件を確認しました")
    print(f"   Cognito User Pool: {user_pool_id}")
    print("   Lab 3B を進める準備ができました")
except Exception as e:
    print("❌ Lab 3A が完了していません。先に Lab-03a-remediation-agent.ipynb を実行してください。")
    raise

---

# ステップ 2: Cognito トークンとグループクレーム

## 目標
Cognito トークンに含まれる内容と、グループクレームがアクセス制御をどのように強制するかを理解します。

## アクティビティ
- [ ] SRE ユーザーの Cognito トークンを取得
- [ ] Approver ユーザーの Cognito トークンを取得
- [ ] JWT トークンをデコード
- [ ] 両方のトークンからグループクレームを表示
- [ ] トークンペイロード間の違いを比較

In [ ]:
# インポート
import json
import boto3
from datetime import datetime
from lab_helpers.parameter_store import get_parameter
from lab_helpers.constants import PARAMETER_PATHS
from lab_helpers.config import AWS_REGION
from lab_helpers.lab_03 import decode_jwt, print_token_claims, compare_tokens

cognito = boto3.client('cognito-idp', region_name=AWS_REGION)

### SRE ユーザートークン

In [ ]:
# SRE ユーザーを認証
user_client_id = get_parameter(PARAMETER_PATHS['cognito']['user_auth_client_id'])
sre_username = get_parameter(PARAMETER_PATHS['cognito']['test_user_email'])
sre_password = get_parameter(PARAMETER_PATHS['cognito']['test_user_password'])

sre_response = cognito.initiate_auth(
    ClientId=user_client_id,
    AuthFlow='USER_PASSWORD_AUTH',
    AuthParameters={'USERNAME': sre_username, 'PASSWORD': sre_password}
)

sre_token = sre_response['AuthenticationResult']['AccessToken']
print(f"✅ SRE を認証しました: {sre_username}")
print(f"   トークン（最初の50文字）: {sre_token}")

In [ ]:
# SRE トークンをデコードして表示
sre_claims = decode_jwt(sre_token)
print_token_claims(sre_claims, "SRE トークンクレーム")

### Approver ユーザートークン

In [ ]:
# Approver ユーザーを認証
approver_username = get_parameter(PARAMETER_PATHS['cognito']['approver_user_email'])
approver_password = get_parameter(PARAMETER_PATHS['cognito']['approver_user_password'])

approver_response = cognito.initiate_auth(
    ClientId=user_client_id,
    AuthFlow='USER_PASSWORD_AUTH',
    AuthParameters={'USERNAME': approver_username, 'PASSWORD': approver_password}
)

approver_token = approver_response['AuthenticationResult']['AccessToken']
print(f"✅ Approver を認証しました: {approver_username}")
print(f"   トークン（最初の50文字）: {approver_token[:50]}...")

In [ ]:
# Approver トークンをデコードして表示
approver_claims = decode_jwt(approver_token)
print_token_claims(approver_claims, "Approver トークンクレーム")

### トークンペイロードの比較

In [ ]:
# トークンを並べて比較
compare_tokens(sre_claims, approver_claims)

### 重要な概念

**JWT 構造:**
- `header.payload.signature`
- ペイロードにはクレーム（ユーザー属性）が含まれる
- 署名がトークンの真正性を検証

**Cognito のグループクレーム:**
- `cognito:groups` がトークンに自動的に含まれる
- ユーザーが所属するすべてのグループをリスト
- カスタム属性不要

**認可フロー:**
```
ユーザー → Cognito → cognito:groups 付きの JWT
                        ↓
                  Gateway がトークンを受信
                        ↓
                Lambda インターセプターが cognito:groups を抽出
                        ↓
                グループを許可されたツールにマッピング
                        ↓
                許可/拒否の決定
```

**権限マッピング（Lambda で実装予定）:**
```python
GROUP_PERMISSIONS = {
    "sre": ["generate_remediation_plan"],
    "approvers": ["execute_remediation_step", "validate_remediation_environment"]
}
```

---

# ステップ 3: インターセプター Lambda のデプロイ

## 目標
Gateway レベルでアクセス制御を強制する Lambda 関数を作成してデプロイします。

## インターセプターロジック
Lambda は受信リクエストを検査し、以下を行います:
- [ ] リクエストヘッダーから JWT を抽出
- [ ] JWT 署名を検証
- [ ] グループクレームをチェック
- [ ] アクションベースのアクセス制御を強制
- [ ] 許可/拒否の決定を返す

### インターセプターコードのレビュー

In [ ]:
# インターセプター Lambda コードをレビュー
with open('lab_helpers/lab_03/interceptor-request.py', 'r') as f:
    print(f.read())

### Lambda 関数のデプロイ

In [ ]:
# インターセプター Lambda を us-west-2 にデプロイ
from lab_helpers.lab_03 import deploy_interceptor
from lab_helpers.parameter_store import put_parameter
from lab_helpers.constants import PARAMETER_PATHS
from lab_helpers.config import AWS_REGION, WORKSHOP_NAME

print("📦 インターセプター Lambda をデプロイ中...")
function_arn = deploy_interceptor(region=AWS_REGION, prefix=WORKSHOP_NAME)

# 後で使用するために保存
put_parameter(PARAMETER_PATHS['lab_03b']['interceptor_function_arn'], function_arn)

print(f"\n✅ インターセプターの準備完了: {function_arn}")

**注意:** インターセプターはステップ 8-10 で、異なるユーザートークンで Gateway を呼び出してテストします。

---

# ステップ 4: Lab 3A Gateway のクリーンアップ

## 目標
Lab 3A の Gateway を削除します。きめ細かいアクセス制御に必要な Lambda インターセプター設定がないためです。

## なぜ Gateway を削除するのか？

**Lab 3A Gateway の設定:**
- 認証不要
- インターセプターなし
- すべてのツールにオープンアクセス

**Lab 3B Gateway の要件:**
- Cognito 経由の JWT 認証
- REQUEST フェーズでの Lambda インターセプター
- ロールベースのアクセス制御強制

**なぜ更新しないのか？**
Gateway インターセプター設定は作成後に変更できません - 初期デプロイ時に設定する必要があります。したがって、以下が必要です:
1. Lab 3A Gateway を削除
2. インターセプター設定付きの新しい Gateway を作成
3. 既存の Runtime を再利用（変更不要）

**削除されるもの:**
- ✅ Gateway（インターセプター設定が必要）
- ✅ Gateway ターゲット（Gateway と一緒に自動削除）

**再利用されるもの:**
- ✅ Runtime（同じエージェントロジック、異なる Gateway）
- ✅ Cognito User Pool（既に設定済み）
- ✅ Lambda インターセプター（ステップ 3 でデプロイ）

### Lab 3A Gateway の削除

In [ ]:
import json
import boto3
import time

# bedrock-agentcore-control クライアントを初期化
agentcore_client = boto3.client('bedrock-agentcore-control', region_name=AWS_REGION)

# すべての Gateway をリスト
print("🔍 すべての Gateway をリスト中...")
response = agentcore_client.list_gateways()
gateways = response.get('items', [])

if not gateways:
    print("ℹ️  Gateway が見つかりません")
else:
    print(f"📋 {len(gateways)} 個の Gateway が見つかりました\n")
    
    # Lab 3A Gateway のみにフィルタリング（Lab 3B で置き換え）
    lab_3a_gateways = [gw for gw in gateways if 'aiml301-remediation-gateway' in gw.get('name', '')]
    
    if not lab_3a_gateways:
        print("ℹ️  削除する Lab 3A Gateway がありません")
    else:
        print(f"🗑️  {len(lab_3a_gateways)} 個の Lab 3A Gateway を削除中\n")
        
        # 各 Lab 3A Gateway を削除
        for gateway in lab_3a_gateways:
            gateway_id = gateway['gatewayId']
            gateway_name = gateway.get('name', 'N/A')
            
            print(f"🗑️  削除中: {gateway_name} ({gateway_id})")
            
            try:
                # まず、すべてのターゲットをリストして削除
                targets_response = agentcore_client.list_gateway_targets(gatewayIdentifier=gateway_id)
                targets = targets_response.get('items', [])
                
                for target in targets:
                    target_id = target['targetId']
                    print(f"   🎯 ターゲットを削除中: {target_id}")
                    agentcore_client.delete_gateway_target(
                        gatewayIdentifier=gateway_id,
                        targetId=target_id
                    )
                
                # すべてのターゲットが削除されるまで待機
                if targets:
                    print(f"   ⏳ ターゲットの削除を待機中...")
                    for _ in range(30):
                        time.sleep(2)
                        check = agentcore_client.list_gateway_targets(gatewayIdentifier=gateway_id)
                        if len(check.get('items', [])) == 0:
                            break
                
                # Gateway を削除
                agentcore_client.delete_gateway(gatewayIdentifier=gateway_id)
                print(f"   ✅ 削除完了")
                
            except Exception as e:
                print(f"   ❌ エラー: {e}")
        
        print(f"\n✅ クリーンアップ完了")

**注意:** Runtime は Lab 3A から再利用 - 削除は不要です。

---

# ステップ 5: JWT 認証付きの新しい Gateway とターゲットを作成

## 目標
JWT 認証と Lambda インターセプター付きの新しい Gateway をデプロイします。

### Gateway 設定

**Gateway 詳細:**
- 名前: `interceptor-gateway-jwt-[ランダム]`
- プロトコル: MCP
- Authorizer タイプ: CUSTOM_JWT
- インターセプトポイント: REQUEST

**JWT 設定:**
```json
{
  "authorizerType": "CUSTOM_JWT",
  "discoveryUrl": "https://cognito-idp.us-west-2.amazonaws.com/us-west-2_POOL_ID/.well-known/openid-configuration",
  "allowedClients": [
    "CLIENT_ID_1",
    "CLIENT_ID_2"
  ]
}
```

**インターセプター設定:**
```json
{
  "interceptionPoints": ["REQUEST"],
  "interceptor": {
    "lambda": {
      "arn": "arn:aws:lambda:us-west-2:ACCOUNT:function:custom-interceptor-request"
    }
  },
  "inputConfiguration": {
    "passRequestHeaders": true
  }
}
```

### Gateway の作成

In [ ]:
import random
import string
import time
import boto3
from lab_helpers.parameter_store import get_parameter, put_parameter
from lab_helpers.constants import PARAMETER_PATHS
from lab_helpers.config import AWS_REGION
from lab_helpers.lab_03.gateway_setup import AgentCoreGatewaySetup

# bedrock-agentcore-control クライアントを初期化
agentcore_client = boto3.client('bedrock-agentcore-control', region_name= AWS_REGION)

# 設定値を取得
user_pool_id = get_parameter(PARAMETER_PATHS['cognito']['user_pool_id'])
user_client_id = get_parameter(PARAMETER_PATHS['cognito']['user_auth_client_id'])
m2m_client_id = get_parameter(PARAMETER_PATHS['cognito']['m2m_client_id'])
interceptor_arn = get_parameter(PARAMETER_PATHS['lab_03b']['interceptor_function_arn'])

# us-west-2 で Gateway IAM ロールを作成
gateway_setup = AgentCoreGatewaySetup(region=AWS_REGION, prefix=WORKSHOP_NAME, verbose=False)
role_info = gateway_setup.create_gateway_service_role()
role_arn = role_info['role_arn']
print(f"✅ Gateway ロール: {role_arn}")

# 一意の Gateway 名を生成
suffix = ''.join(random.choices(string.ascii_lowercase + string.digits, k=10))
gateway_name = f"aiml301-interceptor-gateway-jwt-{suffix}"

print(f"📤 Gateway を作成中: {gateway_name}")
print(f"   Gateway リージョン: us-west-2")
print(f"   Cognito リージョン: {AWS_REGION}")
print(f"   ロール: {role_arn}")
print(f"   インターセプター: {interceptor_arn}")

# Gateway を作成
response = agentcore_client.create_gateway(
    name=gateway_name,
    protocolType="MCP",
    protocolConfiguration={
        "mcp": {
            "supportedVersions": ["2025-03-26"]
        }
    },
    authorizerType="CUSTOM_JWT",
    authorizerConfiguration={
        "customJWTAuthorizer": {
            "discoveryUrl": f"https://cognito-idp.{AWS_REGION}.amazonaws.com/{user_pool_id}/.well-known/openid-configuration",
            "allowedClients": [user_client_id, m2m_client_id]
        }
    },
    interceptorConfigurations=[
        {
            "interceptionPoints": ["REQUEST"],
            "interceptor": {
                "lambda": {"arn": interceptor_arn}
            },
            "inputConfiguration": {
                "passRequestHeaders": True
            }
        }
    ],
    roleArn=role_arn
)

gateway_id = response['gatewayId']
gateway_arn = response['gatewayArn']

print(f"\n✅ Gateway を作成しました: {gateway_id}")
print(f"   ARN: {gateway_arn}")

In [ ]:
# Gateway が READY になるのを待機
print("⏳ Gateway の準備完了を待機中...")
while True:
    response = agentcore_client.get_gateway(gatewayIdentifier=gateway_id)
    
    status = response['status']
    if status == 'READY':
        gateway_url = response['gatewayUrl']
        print(f"✅ Gateway 準備完了: {status}")
        print(f"   URL: {gateway_url}")
        break
    elif status == 'FAILED':
        print(f"❌ Gateway の作成に失敗しました")
        break
    print(f"   ステータス: {status}")
    time.sleep(5)

# 後で使用するために保存
put_parameter(PARAMETER_PATHS['lab_03b']['gateway_id'], gateway_id)
put_parameter(PARAMETER_PATHS['lab_03b']['gateway_url'], gateway_url)

### Gateway 設定の検査

In [ ]:
# Gateway 設定の詳細を表示
result = agentcore_client.get_gateway(gatewayIdentifier=gateway_id)

print("📋 Gateway 設定:")
print("=" * 70)
print(json.dumps(result, indent=2, default=str))
print("=" * 70)

# 主要な設定要素をハイライト
print("\n🔑 主要な設定要素:")
print(f"   Gateway ID: {result['gatewayId']}")
print(f"   Gateway URL: {result['gatewayUrl']}")
print(f"   プロトコル: {result['protocolType']}")
print(f"   Authorizer: {result['authorizerType']}")

# JWT 設定
jwt_config = result['authorizerConfiguration']['customJWTAuthorizer']
print(f"\n🔐 JWT 設定:")
print(f"   Discovery URL: {jwt_config['discoveryUrl']}")
print(f"   許可されたクライアント: {len(jwt_config['allowedClients'])} クライアント")
for i, client in enumerate(jwt_config['allowedClients'], 1):
    print(f"      {i}. {client}")

# インターセプター設定
interceptor_config = result['interceptorConfigurations'][0]
print(f"\n🛡️  インターセプター設定:")
print(f"   Lambda ARN: {interceptor_config['interceptor']['lambda']['arn']}")
print(f"   インターセプションポイント: {interceptor_config['interceptionPoints']}")
print(f"   リクエストヘッダーを渡す: {interceptor_config['inputConfiguration']['passRequestHeaders']}")
print(f"\n   ℹ️  インターセプターは以下を行います:")
print(f"      • Authorization ヘッダーの JWT トークンを検査")
print(f"      • cognito:groups クレームを抽出")
print(f"      • グループベースの権限を強制")
print(f"      • Runtime に到達する前にリクエストを許可/拒否")

インターセプター Gateway から呼び出せるよう Lambda に権限を追加

In [ ]:
# Gateway ARN で Lambda 権限を更新
import boto3

lambda_client = boto3.client('lambda', region_name=AWS_REGION)
function_name = f"{WORKSHOP_NAME}-interceptor-request"
gateway_arn = result['gatewayArn']

print(f"🔧 Gateway ARN で Lambda 権限を更新中...")

# 古い権限を削除
try:
    lambda_client.remove_permission(
        FunctionName=function_name,
        StatementId='AllowGatewayInvoke'
    )
    print(f"   古い権限を削除しました")
except:
    pass

# ソース ARN 付きの新しい権限を追加
lambda_client.add_permission(
    FunctionName=function_name,
    StatementId='AllowGatewayInvoke',
    Action='lambda:InvokeFunction',
    Principal='bedrock-agentcore.amazonaws.com',
    SourceArn=gateway_arn
)

print(f"✅ ソース ARN で Lambda 権限を更新しました")
print(f"   Gateway ARN: {gateway_arn}")

### ターゲット設定

**ターゲット詳細:**
- 名前: `mcp-remediation-target`
- タイプ: MCP Server
- エンドポイント: [ステップ 6 からの Runtime エンドポイント]

**認証情報プロバイダー:**
- タイプ: OAUTH
- プロバイダー: Cognito MCP Provider

### ターゲットの作成

In [ ]:
# Lab 3A から Runtime エンドポイントを取得
runtime_arn = get_parameter(PARAMETER_PATHS['lab_03']['runtime_arn'])
runtime_endpoint = f"https://bedrock-agentcore.{AWS_REGION}.amazonaws.com/runtimes/{runtime_arn.replace(':', '%3A').replace('/', '%2F')}/invocations?qualifier=DEFAULT"
print (f'Runtime エンドポイント: {runtime_endpoint}')
# OAuth プロバイダー ARN を取得
oauth_provider_arn = get_parameter(PARAMETER_PATHS['lab_03']['oauth2_provider_arn'])

# ターゲットを作成
response = agentcore_client.create_gateway_target(
    gatewayIdentifier=gateway_id,
    name="mcp-remediation-target",
    description="JWT 認証付き修復エージェント用の MCP サーバーターゲット",
    targetConfiguration={
        "mcp": {
            "mcpServer": {
                "endpoint": runtime_endpoint
            }
        }
    },
    credentialProviderConfigurations=[
        {
            "credentialProviderType": "OAUTH",
            "credentialProvider": {
                "oauthCredentialProvider": {
                    "providerArn": oauth_provider_arn,
                    "scopes": []
                }
            }
        }
    ]
)

target_id = response['targetId']
print(f"✅ ターゲット作成中: {target_id}")
print(f"   Runtime: {runtime_arn}")

In [ ]:
# ターゲットが READY になるのを待機
print("⏳ ターゲットの準備完了を待機中...")

target_info = agentcore_client.get_gateway_target(
          gatewayIdentifier=gateway_id,
          targetId=target_id
      )
time.sleep(5)
status = target_info.get('status', 'UNKNOWN')

if status == 'READY':
    print(f"✅ ターゲット準備完了")
    print(f"\n🎉 Gateway とターゲットのテスト準備ができました！")
    print(f"   Gateway URL: {gateway_url}")
    print(f"   Target ID: {target_id}")
if status == 'FAILED' or status == 'SYNCHRONIZE_UNSUCCESSFUL':
    print(f"❌ ターゲットがエラー状態: {target_info.get('statusReasons', 'エラーメッセージなし')}")

### ターゲット設定の検査

In [ ]:
# ターゲット設定の詳細を表示
result = agentcore_client.get_gateway_target(
    gatewayIdentifier=gateway_id,
    targetId=target_id
)

print("📋 ターゲット設定:")
print("=" * 70)
print(json.dumps(result, indent=2, default=str))
print("=" * 70)

# 主要な設定要素をハイライト
print("\n🔑 主要な設定要素:")
print(f"   Target ID: {result['targetId']}")
print(f"   ターゲット名: {result['name']}")
print(f"   ステータス: {result['status']}")

# MCP サーバー設定
mcp_config = result['targetConfiguration']['mcp']['mcpServer']
print(f"\n🔌 MCP サーバー設定:")
print(f"   エンドポイント: {mcp_config['endpoint'][:80]}...")
print(f"   ℹ️  Lab 3A の Runtime を参照（再利用）")

# 認証情報プロバイダー設定
cred_config = result['credentialProviderConfigurations'][0]
oauth_config = cred_config['credentialProvider']['oauthCredentialProvider']
print(f"\n🔐 認証情報プロバイダー設定:")
print(f"   タイプ: {cred_config['credentialProviderType']}")
print(f"   プロバイダー ARN: {oauth_config['providerArn']}")
print(f"   ℹ️  マシン間認証に Cognito OAuth プロバイダーを使用")

# フローの説明
print(f"\n📊 リクエストフロー:")
print(f"   1. ユーザーが JWT トークン付きリクエストを送信 → Gateway")
print(f"   2. Gateway が JWT を検証（Cognito OIDC）")
print(f"   3. Lambda インターセプターが権限をチェック")
print(f"   4. 許可された場合 → Gateway がターゲットに転送")
print(f"   5. ターゲットが OAuth を使用して Runtime を認証")
print(f"   6. Runtime がエージェントロジックを実行")
print(f"   7. レスポンスが Gateway を通じて返される")

---

# ステップ 6: MCP Runtime のレビュー

## 目標
Gateway を通じてアクセスする修復エージェント Runtime をレビューします。

### Runtime 設定の確認

In [ ]:
# Lab 3A の Runtime が利用可能か確認
from lab_helpers.parameter_store import get_parameter
from lab_helpers.constants import PARAMETER_PATHS
from lab_helpers.config import AWS_REGION
import boto3

runtime_arn = get_parameter(PARAMETER_PATHS['lab_03']['runtime_arn'])
runtime_id = runtime_arn.split('/')[-1]

print(f"✅ Lab 3A からの Runtime:")
print(f"   ARN: {runtime_arn}")
print(f"   ID: {runtime_id}")
print(f"   リージョン: {AWS_REGION}")

# コントロールプレーンクライアントを使用して Runtime ステータスを確認
agentcore_control = boto3.client('bedrock-agentcore-control', region_name=AWS_REGION)
try:
    runtime_info = agentcore_control.get_agent_runtime(agentRuntimeId=runtime_id)
    status = runtime_info['status']
    print(f"\n📊 Runtime ステータス: {status}")
    
    if status == 'READY':
        print(f"   ✅ Runtime は Gateway 統合の準備ができています")
    elif status == 'FAILED' or status == 'SYNCHRONIZE_UNSUCCESSFUL':
        print(f"❌ ターゲットがエラー状態: {target_info.get('statusReasons', 'エラーメッセージなし')}")
except Exception as e:
    print(f"   ❌ Runtime 確認エラー: {e}")

---

# ステップ 7: Cognito トークンの再生成

## 目標
新しい Gateway に有効な両方のペルソナ用の新しい JWT トークンを取得します。

### SRE ユーザートークン

In [ ]:
# SRE トークンを再生成（ステップ 2 のトークンが期限切れの場合に備えて）
sre_response = cognito.initiate_auth(
    ClientId=user_client_id,
    AuthFlow='USER_PASSWORD_AUTH',
    AuthParameters={'USERNAME': sre_username, 'PASSWORD': sre_password}
)

sre_token = sre_response['AuthenticationResult']['AccessToken']
sre_claims = decode_jwt(sre_token)

print(f"✅ 新しい SRE トークンを生成しました")
print(f"   ユーザー名: {sre_username}")
print(f"   グループ: {sre_claims.get('cognito:groups', [])}")
print(f"   トークン（最初の50文字）: {sre_token[:50]}...")

### Approver ユーザートークン

In [ ]:
# Approver トークンを再生成（ステップ 2 のトークンが期限切れの場合に備えて）
approver_response = cognito.initiate_auth(
    ClientId=user_client_id,
    AuthFlow='USER_PASSWORD_AUTH',
    AuthParameters={'USERNAME': approver_username, 'PASSWORD': approver_password}
)

approver_token = approver_response['AuthenticationResult']['AccessToken']
approver_claims = decode_jwt(approver_token)

print(f"✅ 新しい Approver トークンを生成しました")
print(f"   ユーザー名: {approver_username}")
print(f"   グループ: {approver_claims.get('cognito:groups', [])}")
print(f"   トークン（最初の50文字）: {approver_token[:50]}...")

## 診断情報で Memory からコンテキストを補強

診断情報でコンテキストを補強するために、キュレーションされた Memory から追加情報を取得しましょう。

In [ ]:
agent_memory_client = boto3.client("bedrock-agentcore", region_name=AWS_REGION)

memory_id = get_parameter(PARAMETER_PATHS['memory']['memory_id'])
memory_session_id = get_parameter(PARAMETER_PATHS['memory']['default_session_id'])

print(memory_id)
print(memory_session_id)
actor_id='diagnostics_agent'

  
# エージェント Memory に追加されたイベントをリスト（書き込み成功を確認）
params = {
                "memoryId": memory_id,
                "actorId": actor_id,
                "sessionId": memory_session_id,
                "includePayloads": True
            }
# すべてのメッセージを取得
response = agent_memory_client.list_events(**params)
additional_context=""
for event in response.get("events", []):
    payload = event.get('payload', [])
    for i, item in enumerate(payload):
        if 'conversational' in item:
            text = item['conversational']['content']['text']
            additional_context+=text
additional_context

---

# ステップ 8: SRE がプラン作成をシミュレート

## 目標
SRE ユーザーが修復プランを正常に作成できることを示します。

## シナリオ
SRE が問題を検出し、修復するためのプランを作成します。

### プラン作成の実行

In [ ]:
# SRE トークンで Gateway に接続
from lab_helpers.lab_03.mcp_client import MCPClient

gateway_url = get_parameter(PARAMETER_PATHS['lab_03b']['gateway_url'])

print(f"🔗 SRE ユーザーとして Gateway に接続中...")
print(f"   Gateway: {gateway_url}")
print(f"   ユーザー: {sre_username}")
print(f"   グループ: {sre_claims.get('cognito:groups', [])}")

sre_client = MCPClient(gateway_url, sre_token)
sre_client.initialize()

print(f"\n✅ Gateway に接続しました")

In [ ]:
# 利用可能なツールをリスト
print(f"🔧 利用可能なツールをリスト中...\n")
tools = sre_client.list_tools()

print(f"📋 利用可能なツール（{len(tools)}件）:")
for tool in tools:
    print(f"   • {tool['name']}")
    if 'description' in tool:
        print(tool)
        desc = tool['description']
        print(f"     {desc[:80]}..." if len(desc) > 80 else f"     {desc}")

In [ ]:
# 修復プラン作成ツールを呼び出し
start_time = datetime.now()
print(f"\n🎯 generate_remediation_plan ツールを呼び出し中...\n")
try:
    result = sre_client.call_tool(
        tool_name='mcp-remediation-target___infrastructure_agent',
        arguments={
            'remediation_query': f"""CRM アプリケーションのインフラストラクチャ修復についてサポートが必要です。発生している問題: {additional_context} """,
            'action_type': 'only_plan'
        }
    )
except Exception as e:
    print(f"❌ エラー: {e}")
analysis_time = (datetime.now() - start_time).total_seconds()
print(f"分析時間: {analysis_time:.2f} 秒")
print(f"✅ ツール呼び出し成功！")
print(f"\n📋 修復プラン:")
print(f"{result[:500]}..." if len(result) > 500 else result)

### 成功の確認

In [ ]:
print(f"\n✅ SRE ユーザーが正常に完了:")
print(f"   1. JWT トークンで Gateway に接続")
print(f"   2. 利用可能なツールをリスト")
print(f"   3. generate_remediation_plan ツールを呼び出し")
print(f"   4. 修復プランを受信")
print(f"\n🔒 インターセプターがこの操作を許可した理由:")
print(f"   • ユーザーが 'sre' グループに所属")
print(f"   • ツール 'generate_remediation_plan' は SRE ユーザーに許可されている")

### 期待される結果
✅ SRE トークンでプランが正常に作成されました

---

# ステップ 9: SRE が実行を試行（アクセス拒否）

## 目標
SRE ユーザーが修復を実行できないことをデモンストレーションします - インターセプターがブロックします。

## シナリオ
SRE が作成したプランを実行しようとします。

### 実行の試行

In [ ]:
# 修復実行ツールを呼び出し
print(f"\n🎯 execute_remediation_step ツールを呼び出し中...\n")

result = sre_client.call_tool(
    tool_name='mcp-remediation-target___infrastructure_agent',
    arguments={
            "remediation_query": f"""
            CRM アプリケーションの DynamoDB テーブルの読み取りと書き込み容量をオンデマンドに変更してください。診断情報: {additional_context}
            """,
            "action_type": "only_execute"
        }
)

### アクセス拒否の分析

In [ ]:
# Lambda ログを確認してインターセプターの拒否を確認
# ログが CloudWatch に到達するまで数分かかる場合があります。待機して再試行してください
import boto3
from datetime import datetime

logs = boto3.client('logs', region_name=AWS_REGION)
log_group = '/aws/lambda/aiml301_sre_agentcore-interceptor-request'

streams = logs.describe_log_streams(
    logGroupName=log_group,
    orderBy='LastEventTime',
    descending=True,
    limit=1
)

print("📋 インターセプター Lambda ログ（最後の20行）:\n")
stream = streams['logStreams'][0]
events = logs.get_log_events(
    logGroupName=log_group,
    logStreamName=stream['logStreamName'],
    limit=30,
    startFromHead=False
)

for event in events['events'][-20:]:
    msg = event['message'].strip()
    if any(x in msg for x in ['Tool call', 'User groups', 'not authorized', 'Denying']):
        print(msg)

print("\n✅ インターセプターが SRE ユーザーの execute_remediation_step 呼び出しをブロックしました")

### 期待される結果
❌ アクセス拒否 - インターセプターによる拒否が確認できます

---

# ステップ 10: Approver が実行と検証

## 目標
Approver ユーザーが修復を実行および検証できることを示します。

### パート A: Approver が修復を実行

In [ ]:
approver_client = MCPClient(gateway_url, approver_token)
approver_client.initialize()

In [ ]:
# 修復実行ツールを呼び出し
print(f"\n🎯 execute_remediation_step ツールを呼び出し中...\n")
start_time = time.time()
try:
    result = approver_client.call_tool(
        tool_name='mcp-remediation-target___infrastructure_agent',
        arguments={
            "remediation_query": f"""
            infrastructure_agent とツールへのアクセスを検証するためのテスト呼び出しです。ヘルスチェックを実行してステータスを確認してください。
            """,
            "action_type": "only_execute"
        }
    )
except Exception as e:
    print(f"❌ エラー: {e}")
end_time = time.time()
print(f"  ⏰ 合計所要時間: {end_time - start_time:.2f} 秒")

### パート B: Approver が修正を検証

### 期待される結果
✅ Approver トークンで修復が実行および検証されました

---

# クリーンアップ

## 目標
Lab 3A のベースインフラストラクチャを保持しながら、Lab 3B リソースを削除します。

**削除されるリソース:**
- JWT 認証付きの Gateway
- Lambda インターセプター関数
- Lambda 実行ロール

**保持されるリソース:**
- AgentCore Runtime（Lab 3A から）
- Cognito User Pool とユーザー
- OAuth2 認証情報プロバイダー
- Parameter Store エントリ

In [ ]:
from lab_helpers.lab_03 import cleanup_lab_03b
from lab_helpers.config import AWS_REGION

# Lab 3B リソースをクリーンアップ
#cleanup_lab_03b(region_name=AWS_REGION, verbose=True)

---

# 参考資料
- Lab 3A: 修復エージェントの基礎
- Cognito JWT 設定
- AgentCore Gateway ドキュメント
- Lambda インターセプターパターン